In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 데이터 로드
import json

# 데이터 경로 설정 (Google Drive에서 JSON 파일 경로 지정)
file_path = '/content/drive/MyDrive/small_dataset.json'

# JSON 데이터 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# 데이터 확인
print(f"데이터 개수: {len(dataset)}")
print(dataset[:3])

In [ ]:
import torch
from transformers import BertModel

# 사용자 정의 KoBERT 클래스
class CustomKoBERT(torch.nn.Module):
    def __init__(self):
        super(CustomKoBERT, self).__init__()
        self.bert = BertModel.from_pretrained("monologg/kobert")  # KoBERT 모델
        self.fc_is_immoral = torch.nn.Linear(768, 1)  # 비도덕성 여부 출력 (1개)
        self.fc_intensity = torch.nn.Linear(768, 1)  # 강도 출력 (1개)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]  # [CLS] 토큰에 대한 출력
        is_immoral = torch.sigmoid(self.fc_is_immoral(pooled_output))  # 비도덕성 확률
        intensity = torch.sigmoid(self.fc_intensity(pooled_output)) * 3  # 강도 (1.0 ~ 3.0)
        return is_immoral, intensity

In [ ]:
# 사용자 정의 KoBERT 초기화
kobert_model = CustomKoBERT()

# Google Drive에서 학습된 가중치 로드
kobert_model_path = "/content/drive/MyDrive/kobert_model.pth"
kobert_model.load_state_dict(torch.load(kobert_model_path, map_location=torch.device('cpu')))

# 평가 모드 설정
kobert_model.eval()
print("학습된 KoBERT 모델이 성공적으로 로드되었습니다.")

In [ ]:
from transformers import BertTokenizer

# KoBERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

# KoBERT로 강도와 비도덕성 판단
def predict_immorality_and_intensity(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        is_immoral, intensity_score = kobert_model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
    is_immoral = is_immoral.item() > 0.5  # 비도덕성 여부
    intensity_score = intensity_score.item()  # 강도 점수 (1.0 ~ 3.0)
    return is_immoral, intensity_score

# 테스트
test_text = "너 정말 재수없다."
is_immoral, intensity_score = predict_immorality_and_intensity(test_text)
print(f"입력 텍스트: {test_text}")
print(f"비도덕성 여부: {is_immoral}, 강도 점수: {intensity_score:.2f}")

In [ ]:
# 강도별 응답 정의
responses = {
    "low": [
        "그렇게 생각할 수도 있군요. 하지만 조금 더 긍정적으로 생각해보는 건 어때요?",
        "당신의 의견을 존중하지만, 그런 말을 조심하는 게 좋을 것 같아요."
    ],
    "medium": [
        "그런 표현은 적절하지 않을 수 있어요. 조금 더 신중하게 생각해 주세요.",
        "그 말은 조금 기분 나쁘게 들려요. 다른 사람이 불쾌감을 느낄 수 있는 표현은 피해 주세요."
    ],
    "high": [
        "그런 말은 너무 불쾌합니다. 사과해주세요.",
        "당신은 너무 무례합니다. 예의를 갖추세요."
    ]
}

# 강도 레이블 계산 함수
def get_intensity_label(intensity_score):
    if intensity_score <= 1.9:
        return "low"
    elif 2.0 <= intensity_score <= 2.9:
        return "medium"
    else:
        return "high"

In [ ]:
# KoBERT와 통합된 응답 생성 함수
def generate_response_from_kobert(text):
    is_immoral, intensity_score = predict_immorality_and_intensity(text)

    if not is_immoral:  # 비도덕적이지 않다면
        return "그런 표현은 비도덕적이지 않습니다."

    # 강도 레이블 계산
    intensity_label = get_intensity_label(intensity_score)

    # 강도별 응답 선택
    return responses[intensity_label][0]

In [ ]:
from transformers import pipeline

# KoGPT2 모델 로드
generator = pipeline("text-generation", model="skt/kogpt2-base-v2")

# Few-shot Prompt 작성
def few_shot_prompt(input_text, intensity_label):
    prompt = f"""
다음은 비도덕적인 표현에 대한 적절한 응답 예시입니다.

사용자: 너는 항상 실수만 하는구나.
강도: low
KoGPT2의 응답: 그렇게 느껴질 수도 있겠네요. 하지만 발전을 위해 노력하고 있어요

사용자: 니가 뭘 안다고 나대냐?
강도: medium
KoGPT2의 응답: 그런 표현은 부적절합니다. 차분히 이야기해 주세요.

사용자: 고객 응대도 못하는 버러지 새끼가 무슨 장사를 한다고 설쳐?
강도: high
KoGPT2의 응답: 그런 말은 너무 불쾌합니다. 사과해주세요.

KoGPT2의 응답:"""
    return prompt

In [ ]:
# KoGPT2 응답 생성 및 처리
def generate_gpt2_response(prompt):
    response = generator(prompt, max_new_tokens=50, num_return_sequences=1)
    generated_text = response[0]['generated_text']

    # "KoGPT2의 응답:" 이후 텍스트만 추출
    if "KoGPT2의 응답:" in generated_text:
        response_text = generated_text.split("KoGPT2의 응답:")[1].strip()
    else:
        response_text = generated_text.strip()

    # 두 문장으로 제한
    final_response = limit_to_two_sentences(response_text)
    return final_response

# KoGPT2 응답을 두 문장으로 제한
def limit_to_two_sentences(response):
    sentences = response.split(".")  # "." 기준으로 분리
    limited_response = ".".join(sentences[:2]).strip() + "."  # 두 문장까지 연결
    return limited_response

In [ ]:
# Rule-Based와 KoGPT2 응답 통합
def generate_combined_response(input_text, intensity_label):
    # Rule-Based 응답 생성
    rule_based_response = generate_rule_based_response(intensity_label)

    # Few-shot Prompt 작성
    prompt = few_shot_prompt(input_text, intensity_label)

    # KoGPT2 응답 생성
    try:
        gpt2_response = generate_gpt2_response(prompt)
    except Exception as e:
        print(f"KoGPT2 응답 생성 실패: {e}")
        return rule_based_response  # 실패 시 Rule-Based 응답 반환

    # Rule-Based 응답과 KoGPT2 응답 결합
    combined_response = f"{rule_based_response} 추가적으로, {gpt2_response}"
    return combined_response

In [ ]:
# 테스트 입력
test_input = "너 정말 무례하다."

# KoBERT로 비도덕성과 강도 판단
is_immoral, intensity_score = predict_immorality_and_intensity(test_input)
print(f"입력 텍스트: {test_input}")
print(f"비도덕성 여부: {is_immoral}, 강도 점수: {intensity_score:.2f}")

if is_immoral:
    # 강도 레이블 계산
    intensity_label = get_intensity_label(intensity_score)

    # Prompt 생성
    dynamic_prompt = few_shot_prompt(test_input, intensity_label)

    # KoGPT2 응답 생성
    final_response = generate_gpt2_response(dynamic_prompt)
    print(f"생성된 응답:\n{final_response}")
else:
    print("그런 표현은 비도덕적이지 않습니다.")